basic walk through of package

In [2]:
#!/usr/bin/python
import numpy as np
import scipy.sparse
import pickle
import xgboost as xgb

In [3]:
### simple example
# load file from text file, also binary buffer generated by xgboost
dtrain = xgb.DMatrix('data/agaricus.txt.train')
dtest = xgb.DMatrix('data/agaricus.txt.test')

In [9]:
#help(dtrain)
dtrain.get_label()

array([ 1.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [10]:
# specify parameters via map, definition are same as c++ version
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic' }

# specify validations set to watch performance
watchlist  = [(dtest,'eval'), (dtrain,'train')]
num_round = 2
bst = xgb.train(param, dtrain, num_round, watchlist)

In [12]:
preds = bst.predict(dtest)
labels = dtest.get_label()
print ('error=%f' % ( sum(1 for i in range(len(preds))
if int(preds[i]>0.5)!=labels[i]) /float(len(preds))))

error=0.021726


In [18]:
bst.save_model('0001.model')
# dump model
bst.dump_model('dump.raw.txt')
# dump model with feature map
bst.dump_model('dump.nice.txt','data/featmap.txt')


In [19]:
# save dmatrix into binary buffer
dtest.save_binary('dtest.buffer')
# save model
bst.save_model('xgb.model')
# load model and data in
bst2 = xgb.Booster(model_file='xgb.model')
dtest2 = xgb.DMatrix('dtest.buffer')
preds2 = bst2.predict(dtest2)
# assert they are the same
assert np.sum(np.abs(preds2-preds)) == 0

In [20]:

pks = pickle.dumps(bst2)
# load model and data in
bst3 = pickle.loads(pks)
preds3 = bst3.predict(dtest2)
# assert they are the same
assert np.sum(np.abs(preds3-preds)) == 0

In [21]:
# build dmatrix from scipy.sparse
print ('start running example of build DMatrix from scipy.sparse CSR Matrix')
labels = []
row = []; col = []; dat = []
i = 0
for l in open('data/agaricus.txt.train'):
    arr = l.split()
    print arr
    labels.append(int(arr[0]))
    for it in arr[1:]:
        k,v = it.split(':')
        row.append(i); col.append(int(k)); dat.append(float(v))
    i += 1
csr = scipy.sparse.csr_matrix((dat, (row,col)))
dtrain = xgb.DMatrix(csr, label = labels)
watchlist  = [(dtest,'eval'), (dtrain,'train')]
bst = xgb.train(param, dtrain, num_round, watchlist)

start running example of build DMatrix from scipy.sparse CSR Matrix
['1', '3:1', '10:1', '11:1', '21:1', '30:1', '34:1', '36:1', '40:1', '41:1', '53:1', '58:1', '65:1', '69:1', '77:1', '86:1', '88:1', '92:1', '95:1', '102:1', '105:1', '117:1', '124:1']
['0', '3:1', '10:1', '20:1', '21:1', '23:1', '34:1', '36:1', '39:1', '41:1', '53:1', '56:1', '65:1', '69:1', '77:1', '86:1', '88:1', '92:1', '95:1', '102:1', '106:1', '116:1', '120:1']
['0', '1:1', '10:1', '19:1', '21:1', '24:1', '34:1', '36:1', '39:1', '42:1', '53:1', '56:1', '65:1', '69:1', '77:1', '86:1', '88:1', '92:1', '95:1', '102:1', '106:1', '116:1', '122:1']
['1', '3:1', '9:1', '19:1', '21:1', '30:1', '34:1', '36:1', '40:1', '42:1', '53:1', '58:1', '65:1', '69:1', '77:1', '86:1', '88:1', '92:1', '95:1', '102:1', '105:1', '117:1', '124:1']
['0', '3:1', '10:1', '14:1', '22:1', '29:1', '34:1', '37:1', '39:1', '41:1', '54:1', '58:1', '65:1', '69:1', '77:1', '86:1', '88:1', '92:1', '95:1', '98:1', '106:1', '114:1', '120:1']
['0', '3:

In [ ]:
print ('start running example of build DMatrix from scipy.sparse CSC Matrix')
# we can also construct from csc matrix
csc = scipy.sparse.csc_matrix((dat, (row,col)))
dtrain = xgb.DMatrix(csc, label=labels)
watchlist  = [(dtest,'eval'), (dtrain,'train')]
bst = xgb.train(param, dtrain, num_round, watchlist)

print ('start running example of build DMatrix from numpy array')
# NOTE: npymat is numpy array, we will convert it into scipy.sparse.csr_matrix in internal implementation
# then convert to DMatrix
npymat = csr.todense()
dtrain = xgb.DMatrix(npymat, label = labels)
watchlist  = [(dtest,'eval'), (dtrain,'train')]
bst = xgb.train(param, dtrain, num_round, watchlist)
